In [22]:
import torch
import torchvision
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split


from scipy import stats
    

import math
import random
import pandas as pd


def countDiffMasks(mask1,mask2):
    total_diff=0
    for i in range(len(mask1)):
        m_1=mask1[i].flatten()
        m_2=mask2[i].flatten()
        count_same=(m_1 == m_2).sum()
        count_different=m_1.flatten().shape[0]-count_same
        total_diff+=count_different
    return total_diff


def get_mask_compression(mask_whole_model):
    num_total=0
    num_non_zeros=0
    for mask_each_layer in mask_whole_model:
        num_total+=torch.numel(mask_each_layer)
        num_non_zeros+=torch.count_nonzero(mask_each_layer)
        
    return (num_total-num_non_zeros)/num_total

    
    

def prune_model_get_mask(model,prune_rate):
    '''
    works purely on the model to get
    mask
    '''
    mask_whole_model=[]
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=torch.ones(params.shape)
#             print(nm,params.shape)
            abs_var=torch.std(torch.abs(params.data))
#             print(abs_var)
#             print(params)
            threshold=abs_var*prune_rate
            num_components=params.shape[0]
            for index_component in range(num_components):
                values=params[index_component]            
                re_shaped_values=values.flatten()                
                mask_vals = (torch.abs(re_shaped_values)>threshold).float()
                mask_vals=mask_vals.reshape(values.shape)
#                 print(mask_vals.shape)
                mask_layer[index_component]=mask_vals
            mask_whole_model.append(mask_layer)
    return mask_whole_model
 
    
def get_thresholds_each_layer(model,prune_rate):
    thresholds_per_layer=[]
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=torch.ones(params.shape)
            abs_std=torch.std(torch.abs(params.data))
            threshold=abs_std*prune_rate
            thresholds_per_layer.append(threshold)
    return thresholds_per_layer
    
                
def apply_mask_model(model,list_mask_whole_model):
    mask_layer_count=0
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=list_mask_whole_model[mask_layer_count]
            with torch.no_grad():
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#                 print("Devices are ",params.device,mask_layer.device)
                mask_layer=mask_layer.to(device)
    
                params.data=params.data*mask_layer            
            mask_layer_count+=1
    

def store_weights_in_dic(weight_description,model):
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            if nm not in weight_description:
                weight_description[nm]={}
            num_components=params.shape[0]
            for index_component in range(num_components):
                if index_component not in weight_description[nm]:
                    weight_description[nm][index_component]={}
                values=params[index_component]
                flat_values=values.flatten()
                for index_wt in range(flat_values.shape[0]):
                    if index_wt not in weight_description[nm][index_component]:
                        weight_description[nm][index_component][index_wt]=[]
                    weight_description[nm][index_component][index_wt].append(flat_values[index_wt].detach().item())
    return weight_description


def get_boolean_dict_weight_dict(weight_description,prune_rate,thresholds_per_layer):
    '''
    works on the dictionary of weights
    to create a dict of 1s and 0s to show
    how many times weight is more than threshold
    per layer
    '''
    boolean_weight_description={}
    count=0
    for layer in weight_description.keys():  
#         print("Count = ",count)
        threshold_this_layer=thresholds_per_layer[count]
#         print("Threshold for layer ",count,layer,"is ",threshold_this_layer)
        if layer not in boolean_weight_description:
            boolean_weight_description[layer]={}
        for index_component in weight_description[layer].keys():
            if index_component not in boolean_weight_description[layer]:
                boolean_weight_description[layer][index_component]={}
            for index_wt in weight_description[layer][index_component].keys():
                if index_wt not in boolean_weight_description[layer][index_component]:
                    boolean_weight_description[layer][index_component][index_wt]=[]
                all_wts=weight_description[layer][index_component][index_wt]
                all_wts_boolean=[]
                for wt in all_wts:
                    if abs(wt)>threshold_this_layer:
                        all_wts_boolean.append(1)
                    else:
                        all_wts_boolean.append(0)
                boolean_weight_description[layer][index_component][index_wt]=all_wts_boolean                    
        count+=1
        
    return boolean_weight_description
    
# create mask from boolean weight dictionary
def create_mask_from_boolean_wt(model,boolean_wt_dict):
    mask_whole_model=[]
    for nm, params in model.named_parameters():
        if "weight" in nm and "bn" not in nm and "linear" not in nm:
            mask_layer=torch.ones(params.shape)
#             print(nm,params.shape)
            abs_var=torch.var(torch.abs(params.data))
#             print(abs_var)
#             print(params)
#             threshold=abs_var*prune_rate
            num_components=params.shape[0]
            for index_component in range(num_components):
                values=params[index_component]            
                re_shaped_values=values.flatten() 
                mask_vals=[]
                for val_index in range(re_shaped_values.shape[0]):
                    boolean_vals=boolean_wt_dict[nm][index_component][val_index]
                    m = stats.mode(boolean_vals)
#                     print("Verdict for this weight is ",m[0][0])
                    mask_vals.append(m[0][0])
#                 mask_vals = (torch.abs(re_shaped_values)>threshold).float()                
                mask_vals=np.asarray(mask_vals)
                mask_vals=mask_vals.reshape(values.shape)
#                 print(mask_vals.shape)
                mask_layer[index_component]=torch.from_numpy(mask_vals)
            mask_whole_model.append(mask_layer)
    return mask_whole_model
         

    
# this part copied from shrinkbench

def nonzero(tensor):
    """Returns absolute number of values different from 0

    Arguments:
        tensor {numpy.ndarray} -- Array to compute over

    Returns:
        int -- Number of nonzero elements
    """
    return np.sum(tensor != 0.0)


def model_size(model, as_bits=False):
    """Returns absolute and nonzero model size

    Arguments:
        model {torch.nn.Module} -- Network to compute model size over

    Keyword Arguments:
        as_bits {bool} -- Whether to account for the size of dtype

    Returns:
        int -- Total number of weight & bias params
        int -- Out total_params exactly how many are nonzero
    """

    total_params = 0
    nonzero_params = 0
    for tensor in model.parameters():
        t = np.prod(tensor.shape)
        nz = nonzero(tensor.detach().cpu().numpy())
        if as_bits:
            bits = dtype2bits[tensor.dtype]
            t *= bits
            nz *= bits
        total_params += t
        nonzero_params += nz
    return int(total_params), int(nonzero_params)



def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))        

def correct(output, target, topk=(1,)):
    """Computes how many correct outputs with respect to targets

    Does NOT compute accuracy but just a raw amount of correct
    outputs given target labels. This is done for each value in
    topk. A value is considered correct if target is in the topk
    highest values of output.
    The values returned are upperbounded by the given batch size

    [description]

    Arguments:
        output {torch.Tensor} -- Output prediction of the model
        target {torch.Tensor} -- Target labels from data

    Keyword Arguments:
        topk {iterable} -- [Iterable of values of k to consider as correct] (default: {(1,)})

    Returns:
        List(int) -- Number of correct values for each topk
    """

    with torch.no_grad():
        maxk = max(topk)
        # Only need to do topk for highest k, reuse for the rest
        _, pred = output.topk(k=maxk, dim=1, largest=True, sorted=True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(torch.tensor(correct_k.item()))
        return res


# below copied from shrinkbench, to be used later
# def accuracy(model, dataloader, topk=(1,)):
#     """Compute accuracy of a model over a dataloader for various topk

#     Arguments:
#         model {torch.nn.Module} -- Network to evaluate
#         dataloader {torch.utils.data.DataLoader} -- Data to iterate over

#     Keyword Arguments:
#         topk {iterable} -- [Iterable of values of k to consider as correct] (default: {(1,)})

#     Returns:
#         List(float) -- List of accuracies for each topk
#     """

#     # Use same device as model
#     device = next(model.parameters()).device

#     accs = np.zeros(len(topk))
#     with torch.no_grad():

#         for i, (input, target) in enumerate(dataloader):
#             input = input.to(device)
#             target = target.to(device)
#             output = model(input)

#             accs += np.array(correct(output, target, topk))

#     # Normalize over data length
#     accs /= len(dataloader.dataset)

#     return accs



class CIFAR10CNN(nn.Module):
    """Feedfoward neural network with 1 hidden layer"""
    def __init__(self):
        super(CIFAR10CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc3.is_classifier = True
        


        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        
        return F.log_softmax(x, dim=1)
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        top_1, top_5 = correct(out, labels,topk=(1,5))
#         print("Batch is ",batch[1].shape)
        
        top_1=top_1/batch[1].shape[0]
        top_5=top_5/batch[1].shape[0]

#         print("corr",top_1,top_5)
#         return {'val_loss': loss, 'val_acc': acc}
        return {'val_loss': loss, 'val_acc': acc, 'top_1': top_1, 'top_5': top_5}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        
        batch_top_1s = [x['top_1'] for x in outputs]
#         print(batch_top_1s)
        epoch_top_1 = torch.stack(batch_top_1s).mean()      # Combine top_1
        
        batch_top_5s = [x['top_5'] for x in outputs]
        epoch_top_5 = torch.stack(batch_top_5s).mean()      # Combine top_5
        
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item(),
               'val_top_1': epoch_top_1.item(), 'val_top_5': epoch_top_5.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}, val_top_1: {:.4f}, val_top_5: {:.4f}".format(
                                epoch, result['val_loss'], result['val_acc'], 
                                result['val_top_1'], result['val_top_5']))
        
        


        
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)



class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)
    
    
def evaluate(model, val_loader):
    """Evaluate the model's performance on the validation set"""
    outputs = [model.validation_step(batch) for batch in val_loader]
#     print("outputs are ",outputs)
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD,
        weight_description=None,mask_whole_model=None):
    """Train the model using gradient descent"""
    print("At train")
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            if mask_whole_model:
#                 print("Applying mask")
                apply_mask_model(model,mask_whole_model)
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
        print("wt desc = ",weight_description)
        if weight_description!=None:
            print("going for weight")
            weight_description=store_weights_in_dic(weight_description,model)
    return history, weight_description


def predict_image(img, model):
    xb = to_device(img.unsqueeze(0), device)
    yb = model(xb)
    _, preds  = torch.max(yb, dim=1)
    return preds[0].item()





print("Song Han cifar10")

print("Torch cuda ",torch.cuda.is_available())


device = get_default_device()
print("device ",device)

# data_transforms=torchvision.transforms.Compose(
#     [torchvision.transforms.Resize((28,28)),torchvision.transforms.ToTensor()])


data_transforms=torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()])


dataset = CIFAR10(root='data/', download=True, transform=data_transforms)


# Define test dataset
test_dataset = CIFAR10(root='data/', train=False,transform=data_transforms)



val_size = 10000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

batch_size=128

train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=256)
# test_loader=val_loader

shape=dataset[0][0].shape
input_size=1
for s in shape:
    input_size*=s

    
    



train_loader = DeviceDataLoader(train_loader, device)
val_loader = DeviceDataLoader(val_loader, device)
test_loader = DeviceDataLoader(test_loader, device)

targets=train_ds.dataset.targets
training_data=torch.tensor(train_ds.dataset.data)

training_data = training_data.to(device=device)



model=CIFAR10CNN()
if torch.cuda.is_available():
    model=model.cuda()
    
history = [evaluate(model, val_loader)]
print("initial result",history)
lr=0.01









metrics={}
metrics["prune_rate"]=[]
metrics["compression"]=[]
metrics["epochs"]=[]
metrics["top_5"]=[]
metrics["top_1"]=[]

# df_base=pd.read_csv("results_sheet/song_hn.csv")
# done_pruning_list=list(df_base["prune_rate"])


prune_rate_range=[2.02,2.04,2.06,2.08]
target_compression=0.746

# prune_rate_range=[2.10]

for prune_rate in prune_rate_range:
    print("Prune rate",prune_rate)
#     if prune_rate in done_pruning_list:
#         print("Already done")
#         continue

    model_state_path="model_state/mod_CNN.pt"
    
     
    if torch.cuda.is_available():
        model.load_state_dict(torch.load(model_state_path))
    else:
        model.load_state_dict(torch.load(model_state_path,map_location=torch.device('cpu')))

    result = evaluate(model, test_loader)
    print("Test result is ",result)


    total_size,nz_size=model_size(model)
    compression=(total_size-nz_size)/total_size
    print("Original Compression=",compression)

    
    mask_whole_model=prune_model_get_mask(model,prune_rate)

    print("Mask compression = ",prune_rate,get_mask_compression(mask_whole_model))


    apply_mask_model(model,mask_whole_model)
    total_size,nz_size=model_size(model)
    compression=(total_size-nz_size)/total_size
    res = evaluate(model, test_loader)

    print("After pruning,with",prune_rate," Compression=",compression,"\nResult after pruning is ",res)
    if compression<(target_compression-0.005) or compression>(target_compression+0.005):
        continue
        


    epochs=30
    history_prune,_ = fit(epochs, lr, model, train_loader, val_loader,
                          mask_whole_model=mask_whole_model)


    total_size,nz_size=model_size(model)
    compression=(total_size-nz_size)/total_size
    res = evaluate(model, test_loader)

    print("Compression=",compression,"Result after pruning is ",res)
    metrics["prune_rate"].append(prune_rate)
    metrics["compression"].append(compression)
    metrics["epochs"].append(epochs)
    metrics["top_5"].append(res['val_top_5'])
    metrics["top_1"].append(res['val_top_1'])
    print("*"*20)
    break




Song Han cifar10
Torch cuda  False
device  cpu
Files already downloaded and verified
initial result [{'val_loss': 2.3053929805755615, 'val_acc': 0.10488281399011612, 'val_top_1': 0.10488281399011612, 'val_top_5': 0.503613293170929}]
Prune rate 2.02
Test result is  {'val_loss': 1.215560793876648, 'val_acc': 0.5616210699081421, 'val_top_1': 0.5616210699081421, 'val_top_5': 0.9493163824081421}
Original Compression= 0.0
Mask compression =  2.02 tensor(0.7386)
After pruning,with 2.02  Compression= 0.7357513788988163 
Result after pruning is  {'val_loss': 2.1016793251037598, 'val_acc': 0.18906250596046448, 'val_top_1': 0.18906250596046448, 'val_top_5': 0.7484375238418579}
Prune rate 2.04
Test result is  {'val_loss': 1.215560793876648, 'val_acc': 0.5616210699081421, 'val_top_1': 0.5616210699081421, 'val_top_5': 0.9493163824081421}
Original Compression= 0.0
Mask compression =  2.04 tensor(0.7445)
After pruning,with 2.04  Compression= 0.7416701609521659 
Result after pruning is  {'val_loss': 2.

In [23]:
total_size,nz_size=model_size(model)
compression=(total_size-nz_size)/total_size
res = evaluate(model, test_loader)
print(compression,res)

# add gaussian noise

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

stdevs=np.linspace(0,1,30)

0.7416701609521659 {'val_loss': 1.2146483659744263, 'val_acc': 0.5731445550918579, 'val_top_1': 0.5731445550918579, 'val_top_5': 0.950488269329071}


In [24]:
results={}
results["stdev"]=[]
results["val_1"]=[]
for stdev in stdevs:
    transform_noisy=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        AddGaussianNoise(0., stdev)
    ])
    # Define test dataset
    test_dataset = CIFAR10(root='data/', train=False,transform=transform_noisy)
    test_loader = DataLoader(test_dataset, batch_size=256)
    test_loader = DeviceDataLoader(test_loader, device)

    res = evaluate(model, test_loader)
    
    print(stdev,res)
    results["stdev"].append(stdev)
    results["val_1"].append(res["val_acc"])    


0.0 {'val_loss': 1.2146483659744263, 'val_acc': 0.5731445550918579, 'val_top_1': 0.5731445550918579, 'val_top_5': 0.950488269329071}
0.034482758620689655 {'val_loss': 1.220016598701477, 'val_acc': 0.569531261920929, 'val_top_1': 0.569531261920929, 'val_top_5': 0.9488281011581421}
0.06896551724137931 {'val_loss': 1.238027572631836, 'val_acc': 0.56396484375, 'val_top_1': 0.56396484375, 'val_top_5': 0.9478515386581421}
0.10344827586206896 {'val_loss': 1.2838128805160522, 'val_acc': 0.54736328125, 'val_top_1': 0.54736328125, 'val_top_5': 0.943164050579071}
0.13793103448275862 {'val_loss': 1.3375908136367798, 'val_acc': 0.5331054925918579, 'val_top_1': 0.5331054925918579, 'val_top_5': 0.9375}
0.1724137931034483 {'val_loss': 1.4152674674987793, 'val_acc': 0.510546863079071, 'val_top_1': 0.510546863079071, 'val_top_5': 0.930468738079071}
0.20689655172413793 {'val_loss': 1.522644281387329, 'val_acc': 0.4852539002895355, 'val_top_1': 0.4852539002895355, 'val_top_5': 0.917675793170929}
0.2413793

In [25]:
df_res=pd.DataFrame(results)
df_res

,stdev,val_1
0,0.000000,0.573145
1,0.034483,0.569531
2,0.068966,0.563965
3,0.103448,0.547363
4,0.137931,0.533105
5,0.172414,0.510547
6,0.206897,0.485254
7,0.241379,0.460547
8,0.275862,0.432324
9,0.310345,0.411914


In [26]:
df_res.to_csv("results_sheet/noisy/snhn1.csv",index=False)